In [1]:
from config import *
import concurrent.futures
import random
from glob import glob
from tqdm import tqdm
import numpy as np
import cv2
from skimage import measure, draw
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from collections import Counter
from tqdm import tqdm_notebook as tqdm
import itertools
import seaborn
seaborn.set()

# Data Exploration
In this notebook, we will be examining the dataset by visualizing some examples and performing preliminary analyses. Note that the dataset used for training the models was not limited to APTOS 2019 Blindness Detection competition and used the previous Diabetic Retinopathy Detection competition as well.

## Table of Contents
1. [Class statistics](#Class-statistics)
2. [Visualizing the data](#Visualizing-the-data)
3. [Image statistics](#Image-statistics)

In [2]:
plot = False

## Class statistics

In [3]:
labels = {}
dist = {}

### APTOS

Let's begin by inspecting the labels in the APTOS dataset. These and all the other datasets contain the diagnosis information which is in the following integer units.

0 - No DR

1 - Mild

2 - Moderate

3 - Severe

4 - Proliferative DR

Note: DR = Diabetic Retinopathy

In [4]:
labels['aptos'] = pd.read_csv(os.path.join( DATA_DIR['aptos'], 'train', 'labels.csv'))
if plot:
    display(labels['aptos'].head(10))

We can check the class distribution and plot it.

In [5]:
dist['aptos'] = labels['aptos'].groupby(['diagnosis']).count()
dist['aptos'].columns = ['count']
if plot:
    _ = dist['aptos'] .plot(kind = 'bar', width = 1, title='APTOS Distribution')

The majority of the photos contain `No DR`. To overcome the negative influences of this imbalance, we will need to employ measures later.
Now we will inspect the class distributions for the external datasets. Note that we have access to the test labels for them as well.

### CHF

In [6]:
labels['chf'] = pd.concat((pd.read_csv(os.path.join( DATA_DIR['chf'], 'train', 'labels.csv')),
                 pd.read_csv(os.path.join( DATA_DIR['chf'], 'test', 'labels.csv'))))
if plot:
    display(labels['chf'].head(10))
dist['chf'] = labels['chf'].groupby(['diagnosis']).count()
dist['chf'].columns = ['count']
if plot:
    _ = dist['chf'].plot(kind = 'bar', width = 1, title='CHF Distribution')

The imbalance is even more pronounced in this larger dataset.

### IDRiD

In [7]:
labels['idrid'] = pd.concat((pd.read_csv(os.path.join( DATA_DIR['idrid'], 'grading', 'train', 
                                           'labels.csv')),
                             pd.read_csv(os.path.join( DATA_DIR['idrid'], 'grading', 'test', 
                                           'labels.csv'))))
if plot:
    display(labels['idrid'].head(10))
dist['idrid'] = labels['idrid'].groupby(['diagnosis']).count()
dist['idrid'].columns = ['count']
if plot:
    _ = dist['idrid'].plot(kind = 'bar', width = 1, title='IDRiD Distribution')

Now this is much better but notice the lower counts. 
### Aggregate
Let's now look at the aggregate distribution as we will be most likely using all of these datasets.

In [8]:
dist['agg'] = sum([dist[ds] for ds in ['aptos', 'chf', 'idrid']])
if plot:
    _=dist['agg'].plot(kind = 'bar', width = 1, title='Aggregate Distribution')

In [9]:
if plot:
    _=plt.pie(dist['agg']['count'], labels=dist['agg'].index, autopct='%.1f')

Overall, this is a highly imbalanced dataset. Also it is worth pointing out that with these external datasets we are expanding the size of our training data by about **24 folds**: 

In [10]:
int((dist['agg'].sum() - dist['aptos'].sum())/ dist['aptos'].sum())

24

## Visualizing the data

But wait. Are the images for these datasets comparable enough to aggregate them in the first place? Let's look and make a guess.

In [11]:
def plot_images(paths, labels, imgs_to_show):
    fig, axes = plt.subplots(int(imgs_to_show**.5), int(imgs_to_show**.5), figsize=(15, 15), facecolor='black')
    for r in range(len(axes)):
        for c in range(len(axes[0])):
            path = paths[r * 4 + c]
            image_id = path[path.find('img/')+4:-4]
            diagnosis = labels[labels['id'] == image_id]['diagnosis']
            axes[r, c].imshow(plt.imread(path))
            axes[r, c].set_title('{}'.format(int(diagnosis)), color='w', style='oblique')
            axes[r, c].axis('off')
    return fig

In [12]:
paths = {}
imgs_to_show = 36

### APTOS

In [13]:
ds = 'aptos'

In [14]:
paths[ds] = glob(os.path.join(DATA_DIR[ds], 'train', 'img', '*.png'))
random.shuffle(paths[ds])

In [15]:
if plot:
    plot_images(paths[ds], labels[ds], imgs_to_show)

Note that the labels above the images are the diagnosis values. From inspecting a small subset of the APTOS images, we can see that there are several issues to pay attention to.

* The ligthing conditions are not constant, which causes dramatic changes in average colors. 
* Some eyes are not fully in the image, they are clipped from top and bottom.
* Left and right eyes are noticeable and different.

### CHF

In [16]:
ds = 'chf'

In [17]:
paths[ds] = glob(os.path.join(DATA_DIR[ds], 'train', 'img', '*.jpeg'))
random.shuffle(paths[ds])

In [18]:
if plot:
    plot_images(paths[ds], labels[ds], imgs_to_show)

The lighting othe CHF dataset appears to be less consistent. Same issues apply to here as well. 

### IDRiD

In [19]:
ds = 'idrid'

In [20]:
paths[ds] = glob(os.path.join(DATA_DIR[ds], 'grading', 'train', 'img', '*.jpg'))
random.shuffle(paths[ds])

In [21]:
if plot:
    plot_images(paths[ds], labels[ds], imgs_to_show)

Overall, IDRiD dataset appears to have the most consistent data.

## Image statistics

Another useful exploration is to actually quantify what we have done in the previous step. So, instead of guessing we will explore some quantifiable measures of the datasets.

I find the following measures interesting for the challenge:

* Mean/variance width and height of the image
* Bounding region of the eye
* Mean/variance of color of eye region (3 channel)
* \# of clipped vs full eye

In [22]:
stats = {}
template = {'width':[], 'height': [], 'blue': [], 'green': [], 'red': [], 'clipped_h':0, 'clipped_w':0}
plt.rcParams['axes.grid'] = False
plt.rcParams['figure.max_open_warning'] = -1

In [23]:
def fill_mask(mask):
    h, w = mask.shape
    floodfill = np.zeros((h+2, w+2), dtype=np.uint8)
    floodfill[1:-1, 1:-1] = mask
    floodfill_mask = np.zeros((h+4, w+4), dtype=np.uint8)
    cv2.floodFill(floodfill, floodfill_mask, (0,0), 1)
    floodfill_inv = cv2.bitwise_not(floodfill)[1:-1, 1:-1]
    final_mask = mask | floodfill_inv
    return final_mask

In [25]:
def collect_stats(path, return_img = False, return_mask = False):
    stats = {'width':[], 'height': [], 'blue': [], 'green': [], 'red': [], 'clipped_h':0, 'clipped_w':0}
    img = cv2.imread(path, cv2.IMREAD_UNCHANGED)
    h, w, _ = img.shape
    mask = find_eye(img)
    eye_hh, eye_hw = np.where(mask)
    eye_ww, eye_wh = np.where(mask.T)
    clipped_h = eye_hh[0] == 0 or eye_hh[-1] == h-1
    clipped_w = eye_ww[0] == 0 or eye_ww[-1] == w-1
    blue, green, red = np.mean(img[np.where(mask)], axis=0)

    stats['blue'].append(blue)
    stats['green'].append(green)
    stats['red'].append(red)
    stats['height'].append(h)
    stats['width'].append(w)
    stats['clipped_h'] = clipped_h
    stats['clipped_w'] = clipped_w
    if return_mask:
        if return_img:
            return stats, img, mask
        return stats, mask
    if return_img:
        return stats, img
    return stats

In [26]:
def merge_dicts(dicts):
    final_dict = {}
    for key in template:
        if type(template[key]) == list:
            final_dict[key] = list(itertools.chain.from_iterable([d[key] for d in dicts]))
        else:
            final_dict[key] = sum([d[key] for d in dicts])
    final_dict['N'] = len(dicts)
    return final_dict

In [27]:
def report(stats):
    fig, axes = plt.subplots(1, 2, sharey=True, figsize=(10, 5))
    axes[0].set_title('Height')
    axes[0].set_xticks(np.arange(400, 3901, 500))
    axes[0].hist(stats['height'], bins=np.arange(400, 3901, 250))
    axes[1].set_title('Width')
    axes[1].set_xticks(np.arange(400, 3901, 500))
    _ = axes[1].hist(stats['width'], bins=np.arange(400, 3901, 250))
    print('mean height: {:.2f}'.format(sum(stats['height'])/stats['N']))
    print('mean width: {:.2f}'.format(sum(stats['width'])/stats['N']))
    
    fig, axes = plt.subplots(1, 3, sharey=True, figsize=(15, 5))
    axes[0].set_title('Blue')
    axes[0].set_xticks(np.arange(0, 256, 32))
    axes[0].hist(stats['blue'], bins=np.arange(0, 256, 16))

    axes[1].set_title('Green')
    axes[1].set_xticks(np.arange(0, 256, 32))
    axes[1].hist(stats['green'], bins=np.arange(0, 256, 16))

    axes[2].set_title('Red')
    axes[2].set_xticks(np.arange(0, 256, 32))
    _ = axes[2].hist(stats['red'], bins=np.arange(0, 256, 16))

    stats['mean_blue'] = mean_blue = sum(stats['blue'])/stats['N']
    stats['std_blue'] = std_blue = np.std(stats['blue'])
    print('mean blue: {:.2f}, std blue: {:.2f}'.format(mean_blue, std_blue))

    stats['mean_green'] = mean_green = sum(stats['green'])/stats['N']
    stats['std_green'] = std_green = np.std(stats['green'])
    print('mean green: {:.2f}, std green: {:.2f}'.format(mean_green, std_green))

    stats['mean_red'] = mean_red = sum(stats['red'])/stats['N']
    stats['std_red'] = std_red = np.std(stats['red'])
    print('mean red: {:.2f}, std red: {:.2f}'.format(mean_red, std_red))
    px = np.round([mean_red, mean_green, mean_blue]).astype(np.uint8)[np.newaxis, np.newaxis, :]
    im = np.zeros((10, 10, 3), dtype=np.uint8)
    im = im + px 
    fig, axes = plt.subplots(1, 1, figsize=(15, 5))
    fig.suptitle('Average color of the eye')
    _=axes.imshow(im)
    print('{:.2%} is clipped in height'.format(stats['clipped_h']/ stats['N']))
    print('{:.2%} is clipped in width'.format(stats['clipped_w']/ stats['N']))